
# TP1 : Web scraping et APIs – Correction

Cette version corrigée du TP1 répond aux consignes du devoir :

* télécharger automatiquement les 51 fichiers PDF du corpus CAMille via du web scraping ;
* choisir et interroger une API utile pour enrichir le corpus (ici l'API [Genderize](https://api.genderize.io/) qui prédit le genre probable d'un prénom)【688134311751931†L0-L0】 ;
* documenter chaque étape et utiliser Python et BeautifulSoup.



In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup



In [ ]:
# URL racine contenant les 51 fichiers PDF du corpus CAMille
root_url = 'https://max.de.wilde.web.ulb.be/camille/'

# Envoi de la requête avec un User-Agent pour éviter les blocages
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(root_url, headers=headers)
response.raise_for_status()

# Parsing du HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Récupération de tous les liens vers les fichiers PDF
links = soup.find_all('a')
pdf_links = []
for link in links:
    href = link.get('href')
    if href and href.lower().endswith('.pdf'):
        # Construction de l'URL absolue
        pdf_links.append(root_url + href)

print(f"Nombre de fichiers PDF trouvés : {len(pdf_links)}")
print('Les cinq premiers liens :')
for l in pdf_links[:5]:
    print(l)



In [ ]:
# Création du dossier de sortie pour les PDF
output_dir = 'data/camille_pdfs'
os.makedirs(output_dir, exist_ok=True)

# Téléchargement de chaque PDF avec gestion des doublons
for idx, link in enumerate(pdf_links, 1):
    filename = link.split('/')[-1]
    out_path = os.path.join(output_dir, filename)
    if os.path.exists(out_path):
        print(f"{idx}/{len(pdf_links)} - {filename} déjà présent, on passe.")
        continue
    try:
        print(f"{idx}/{len(pdf_links)} - Téléchargement de {filename} ...")
        pdf_resp = requests.get(link, headers=headers)
        pdf_resp.raise_for_status()
        with open(out_path, 'wb') as f:
            f.write(pdf_resp.content)
        time.sleep(0.5)  # pause légère pour ne pas saturer le serveur
    except Exception as e:
        print(f"Erreur lors du téléchargement de {filename}: {e}")

print('Téléchargements terminés.')



## Choix d'une API pour enrichir le corpus

Plutôt que d'utiliser l'API initialement sélectionnée mais désormais obsolète, nous avons choisi l'API **Genderize** qui, à partir d'un prénom, renvoie un genre probable (masculin ou féminin) ainsi qu'une probabilité associée. Cette API est simple d'utilisation et peut enrichir le corpus en indiquant le genre probable des personnes citées dans les articles.

Par exemple, une requête sur l'URL `https://api.genderize.io/?name=alice` renvoie un objet JSON contenant le prénom, le genre estimé, la probabilité et un compteur de corpus【688134311751931†L0-L0】.


In [ ]:
def predict_gender(name):
    # Appel à l'API Genderize pour un prénom donné
    url = 'https://api.genderize.io/'
    params = {'name': name}
    try:
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        return data.get('gender'), data.get('probability')
    except Exception as e:
        print(f"Erreur lors de la requête pour {name}: {e}")
        return None, None

# Exemples d'utilisation
sample_names = ['Jean', 'Marie', 'Pierre', 'Alice']
for name in sample_names:
    gender, prob = predict_gender(name)
    print(f"{name} -> genre : {gender}, probabilité : {prob}")



## Conclusion

Ce notebook télécharge automatiquement les 51 fichiers PDF du corpus CAMille et illustre l'utilisation d'une API externe pour enrichir le corpus. Vous pouvez adapter la fonction `predict_gender` à des listes de prénoms extraits des articles pour obtenir une annotation du genre probable des personnes mentionnées.
